# **ТЕОРИЯ**

**SMT** - статистический машинный перевод, в котором используются параллельные корпуса. Выдает наиболее вероятный перевод фразы путем сопоставления параллельных корпусов. Учит соответствия между словами и фразами в разных языках на основе частоты их совместного появления.

**Этапы работы модели**:
1. делаем вероятностное распределение конструкций на уровне слов или фраз
2. собираем статистику соответствий в параллельном корпусе, определяем соответствия, выбираем лучшие при помощи теор. вер.
3. отбираем лучшую версию, отсекая плохие по грамматике и лексике

**Где взять корпуса:**

*   OPUS Corpora
*   Kaggle
*   HuggingFace





# **МОДЕЛЬ**

## **Загружаем необходимые библиотеки**

In [17]:
import tarfile

from sklearn.model_selection import train_test_split

from collections import Counter, defaultdict
import random

## **Подготавливаем данные**

Загружаем датасет

In [42]:
# извлекаем файлы из архива
with tarfile.open('de_en_tiny.tar', 'r') as tar:
  tar.extractall()

Смотрим, что  распаковали

In [43]:
!ls

de-en.tiny.de  de-en.tiny.en  de_en_tiny.tar  sample_data


Создаем 2 выборки и токенизируем тексты по фразам

In [46]:
with open('de-en.tiny.de', 'r') as f:
  german = f.read().split('\n')[:-1]

with open('de-en.tiny.en', 'r') as f:
  english = f.read().split('\n')[:-1]

print("Данные языка X:\n", german)
print("Данные языка Y:\n", english)

Данные языка X:
 ['Wiederaufnahme der Sitzungsperiode', 'Ich bitte Sie , sich zu einer Schweigeminute zu erheben .', '( Das Parlament erhebt sich zu einer Schweigeminute . )', 'Frau Präsidentin , zur Geschäftsordnung .', 'Wenn das Haus damit einverstanden ist , werde ich dem Vorschlag von Herrn Evans folgen .', 'Frau Präsidentin , zur Geschäftsordnung .', 'Könnten Sie mir eine Auskunft zu Artikel 143 im Zusammenhang mit der Unzulässigkeit geben ?', 'Und zwar sollen derartige Strafen trotz des Grundsatzes der relativen Stabilität verhängt werden .', 'All dies entspricht den Grundsätzen , die wir stets verteidigt haben .', 'Vielen Dank , Herr Segni , das will ich gerne tun .', 'Das ist ganz im Sinne der Position , die wir als Parlament immer vertreten haben .', 'Das ist der Fall von Alexander Nikitin .', 'Nun ist es aber so , daß er wieder angeklagt werden soll , weil der Staatsanwalt in Berufung geht .', 'Dennoch , Frau Präsidentin , wurde meinem Wunsch nicht entsprochen .', 'Deshalb mö

Разделяем выборку с помощью sklearn

In [47]:
X_train, X_test, y_train, y_test = train_test_split(english, german)

print("> Обучающая выборка:")
for text, label in zip(X_train, y_train):
    print(f"\nТекст на немецком: {label}\n Его перевод на английский: {text}\n")

print("> Тестовая выборка:")
for text, label in zip(X_test, y_test):
    print(f"\nТекст на немецком: {label}\n Его перевод на английский: {text}\n")

> Обучающая выборка:

Текст на немецком: Es liefert eine gute Diskussionsgrundlage und ist insofern zu begrüßen .
 Его перевод на английский: It forms a sound basis for discussion and is to be welcomed in this sense .


Текст на немецком: Vielen Dank .
 Его перевод на английский: Thank you very much .


Текст на немецком: Jetzt sind wir dadurch sehr in Verzug geraten .
 Его перевод на английский: We are badly behind now in this matter .


Текст на немецком: So weit meine Ausführungen zur ersten Priorität , der Wettbewerbsfähigkeit auf regionaler Ebene .
 Его перевод на английский: So that is the first priority , the competitiveness of regional economies .


Текст на немецком: Wir wissen , daß es den jeweiligen Staaten obliegt , einen Teil der Mittel selbst zu verteilen .
 Его перевод на английский: As you know , it is up to each State to redistribute part of the total appropriation .


Текст на немецком: Der Wettbewerb zwischen den Regionen wird sicherlich die Europäische Union stärken

Чистим данные

In [48]:
def tokenize(sentences):
  # функция возвращает списки слов
  return [sentence.split() for sentence in sentences]

# токенизируем каждую выборку
X_train_tokens, X_test_tokens, y_train_tokens, y_test_tokens = tokenize(X_train), tokenize(X_test), tokenize(y_train), tokenize(y_test)

print('Образец токенизированного текста:', X_train_tokens)

Образец токенизированного текста: [['It', 'forms', 'a', 'sound', 'basis', 'for', 'discussion', 'and', 'is', 'to', 'be', 'welcomed', 'in', 'this', 'sense', '.'], ['Thank', 'you', 'very', 'much', '.'], ['We', 'are', 'badly', 'behind', 'now', 'in', 'this', 'matter', '.'], ['So', 'that', 'is', 'the', 'first', 'priority', ',', 'the', 'competitiveness', 'of', 'regional', 'economies', '.'], ['As', 'you', 'know', ',', 'it', 'is', 'up', 'to', 'each', 'State', 'to', 'redistribute', 'part', 'of', 'the', 'total', 'appropriation', '.'], ['Competition', 'between', 'the', 'regions', 'will', 'certainly', 'strengthen', 'rather', 'than', 'weaken', 'the', 'European', 'Union', '.'], ['For', 'us', 'new', 'members', ',', 'it', 'was', 'the', 'first', 'time', ',', 'and', 'this', 'was', 'a', 'very', 'interesting', 'process', '.'], ['In', 'addition', ',', 'the', 'need', 'for', 'greater', 'transparency', 'has', 'been', 'pointed', 'out', '.'], ['Mr', 'President', ',', 'a', 'White', 'Paper', ',', 'by', 'definition

Создаем словарь уникальных словоформ

In [49]:
x_vocab = Counter(' '.join(german).split()).keys()
y_vocab = Counter(' '.join(english).split()).keys()

print(f"Словарь немецких словоформ: {x_vocab}\n Всего {len(x_vocab)} словоформ")
print(f"\nCловарь английских словоформ: {y_vocab}\n Всего {len(y_vocab)} словоформ")

Словарь немецких словоформ: dict_keys(['Wiederaufnahme', 'der', 'Sitzungsperiode', 'Ich', 'bitte', 'Sie', ',', 'sich', 'zu', 'einer', 'Schweigeminute', 'erheben', '.', '(', 'Das', 'Parlament', 'erhebt', ')', 'Frau', 'Präsidentin', 'zur', 'Geschäftsordnung', 'Wenn', 'das', 'Haus', 'damit', 'einverstanden', 'ist', 'werde', 'ich', 'dem', 'Vorschlag', 'von', 'Herrn', 'Evans', 'folgen', 'Könnten', 'mir', 'eine', 'Auskunft', 'Artikel', '143', 'im', 'Zusammenhang', 'mit', 'Unzulässigkeit', 'geben', '?', 'Und', 'zwar', 'sollen', 'derartige', 'Strafen', 'trotz', 'des', 'Grundsatzes', 'relativen', 'Stabilität', 'verhängt', 'werden', 'All', 'dies', 'entspricht', 'den', 'Grundsätzen', 'die', 'wir', 'stets', 'verteidigt', 'haben', 'Vielen', 'Dank', 'Herr', 'Segni', 'will', 'gerne', 'tun', 'ganz', 'Sinne', 'Position', 'als', 'immer', 'vertreten', 'Fall', 'Alexander', 'Nikitin', 'Nun', 'es', 'aber', 'so', 'daß', 'er', 'wieder', 'angeklagt', 'soll', 'weil', 'Staatsanwalt', 'in', 'Berufung', 'geht', 'D

## **Выбираем лучшие соответствия**

### Создаем статистику соответствий
IBM 1 Expectation-Maximization (t-model)

In [83]:
# вероятность того, что случайное слово x_vocab соответсвует случайному слову y_vocab
uniform = 1 / (len(x_vocab) * len(y_vocab))

round(uniform, 8)

5.7e-07

In [84]:
# t-model
t = {}

for i in range(len(X_train)):
  # начинаем итерацию по обучающей выборке
  for word_x in X_train_tokens[i]:
    for word_y in y_train_tokens[i]:
      # создаем t-table
      t[(word_x, word_y)] = uniform

# t-table
for elem in t:
  print("Соответствие |", elem[0], "  ->  ", elem[1], "| Вероятность:", round(t[elem], 3))

Выходные данные были обрезаны до нескольких последних строк (5000).
Соответствие | despite   ->   verhängt | Вероятность: 0.0
Соответствие | despite   ->   werden | Вероятность: 0.0
Соответствие | despite   ->   . | Вероятность: 0.0
Соответствие | the   ->   zwar | Вероятность: 0.0
Соответствие | the   ->   sollen | Вероятность: 0.0
Соответствие | the   ->   derartige | Вероятность: 0.0
Соответствие | the   ->   Strafen | Вероятность: 0.0
Соответствие | the   ->   trotz | Вероятность: 0.0
Соответствие | the   ->   Grundsatzes | Вероятность: 0.0
Соответствие | the   ->   relativen | Вероятность: 0.0
Соответствие | the   ->   Stabilität | Вероятность: 0.0
Соответствие | the   ->   verhängt | Вероятность: 0.0
Соответствие | principle   ->   Und | Вероятность: 0.0
Соответствие | principle   ->   zwar | Вероятность: 0.0
Соответствие | principle   ->   sollen | Вероятность: 0.0
Соответствие | principle   ->   derartige | Вероятность: 0.0
Соответствие | principle   ->   Strafen | Вероятность:

In [85]:
# количество итераций обучения
epochs = 7

In [86]:
for epoch in range(epochs):
  # начинаем обучение

  # шаг 0. создаем слоты для подсчета статистики
  count = {} # P(x|y)
  total = {} # P(y)

  for i in range(len(X_train)):
    # начинаем итерацию по обучающей выборке
    for word_x in X_train_tokens[i]:
      for word_y in y_train_tokens[i]:
        # создаем слоты для подсчета условной вероятности совпадений в корпусе
        count[(word_x, word_y)] = 0
        # и слоты для статистической языковой модели y
        total[word_y] = 0

  # шаг 1. Expectation
  for i in range(len(X_train)):
    # начинаем итерацию по обучающей выборке
    total_stat = {} # статистика x

    # собираем предварительную статистику на основе данных x
    for word_x in X_train_tokens[i]:
      total_stat[word_x] = 0 # создаем слоты для подсчета статистики по каждому токену x
      for word_y in y_train_tokens[i]:
        # обновляем данные из t-table; увеличиваем значения при обнаружении совместной встречаемости
        total_stat[word_x] += t[(word_x, word_y)]

    # обновляем данные для P(x|y) и P(y)
    for word_x in X_train_tokens[i]:
      for word_y in y_train_tokens[i]:
        # подсчет условной вероятности совпадений в корпусе: равномерное распределение / частотность x
        count[(word_x, word_y)] += t[(word_x, word_y)] / total_stat[word_x]
        # подсчет статистической информации y: равномерное распределение / частотность x
        total[word_y] += t[(word_x, word_y)] / total_stat[word_x]

  # шаг 2. Maximization
  for i in range(len(X_train)):
    # начинаем итерацию по обучающей выборке
    for word_x in X_train_tokens[i]:
      for word_y in y_train_tokens[i]:
        # обновляем t-table: вероятность совпадения в корпусе / вероятность информации y
        t[(word_x, word_y)] = count[(word_x, word_y)] / total[word_y]

for elem in t:
  print("Соответствие |", elem[0], "  ->  ", elem[1], "| Вероятность:", round(t[elem], 3))

Выходные данные были обрезаны до нескольких последних строк (5000).
Соответствие | despite   ->   verhängt | Вероятность: 0.09
Соответствие | despite   ->   werden | Вероятность: 0.0
Соответствие | despite   ->   . | Вероятность: 0.0
Соответствие | the   ->   zwar | Вероятность: 0.01
Соответствие | the   ->   sollen | Вероятность: 0.01
Соответствие | the   ->   derartige | Вероятность: 0.01
Соответствие | the   ->   Strafen | Вероятность: 0.01
Соответствие | the   ->   trotz | Вероятность: 0.01
Соответствие | the   ->   Grundsatzes | Вероятность: 0.01
Соответствие | the   ->   relativen | Вероятность: 0.01
Соответствие | the   ->   Stabilität | Вероятность: 0.01
Соответствие | the   ->   verhängt | Вероятность: 0.01
Соответствие | principle   ->   Und | Вероятность: 0.0
Соответствие | principle   ->   zwar | Вероятность: 0.09
Соответствие | principle   ->   sollen | Вероятность: 0.09
Соответствие | principle   ->   derartige | Вероятность: 0.09
Соответствие | principle   ->   Strafen |

### Биграммная модель

In [87]:
# для обучения модели объединим 2 выборки
tokens = ' '.join(german).split()

# хранилище для биграмм
bigram_model = defaultdict(list)

# собираем все попарные совпадения
for i in range(len(tokens)-1):
    current_word = tokens[i]
    next_word = tokens[i + 1]
    bigram_model[current_word].append(next_word)

print(bigram_model)

def decoder(model, steps=5):
  # инициализация случайного токена
  current_word = random.choice(tokens)
  generated_sentence = current_word

  for step in range(steps):
    # пошаговая генерация
    print('Шаг', step+1)
    next_word_options = model[current_word]
    print(f'Правдоподобные варианты продолжения для токена {current_word}:', next_word_options)

    current_word = random.choice(next_word_options)
    generated_sentence += ' '
    generated_sentence += current_word
    print('Промежуточный результат:', generated_sentence)
    print()
  print('Результат:', generated_sentence)

decoder(bigram_model)

defaultdict(<class 'list'>, {'Wiederaufnahme': ['der'], 'der': ['Sitzungsperiode', 'Unzulässigkeit', 'relativen', 'Position', 'Fall', 'Staatsanwalt', 'Arbeitsschutzausschuß', 'Fall', 'dortigen', 'großen', 'EU-Mitgliedstaaten', 'EU', 'Lage', 'PSE-Fraktion', 'Konferenz', 'Präsidenten', 'Konferenz', 'Präsidenten', 'Beschluß', 'Sozialistischen', 'PPE-DE-Fraktion', 'Tagesordnung', 'Fraktion', 'Tagesordnung', 'gegen', 'Werbetrommel', 'Abstimmung', 'Sitzung', 'Subsidiarität', 'ersten', 'Verkehrssicherheit', 'Prüfungsvorschriften', 'Kommission', 'Straße', 'Anwendung', 'technischen', 'Transportbehälter', 'Kommission', 'morgigen', 'Kommission', 'Zusätzlichkeit', 'Kommission', 'Wirtschaftsstruktur', 'Sozialwirtschaft', 'Arbeitsplätze', 'Union', 'fragwürdigen', 'Konservativen', 'entsprechenden', 'Leitlinien', 'Erarbeitung', 'Leitlinien', 'Evaluierung', 'Ergebnisse', 'Bericht', 'Städte', 'nationalen', 'regionalen', 'Mittel', 'Politik', 'Mittel', 'Mittelauszahlung', 'Mitgliedstaaten', 'Leitlinien', 

## **Проверяем результат**

Выбираем наиболее правдоподобное предложение

In [109]:
# сортировка t-table по убыванию правдоподобия
sorted_t = sorted(t.items(), key = lambda k:(k[1], k[0]), reverse = True)

def translate(token):
  for element in sorted_t:
    if element[0][1] == token:
      # поиск совпадений в t-table
      return element[0][0]
  return ''  # Возвращаем пустую строку, если перевод не найден

for sentence in y_test_tokens:
  print("Оригинальное предложение:", ' '.join(sentence))
  translation = []
  for token in sentence:
    translation.append(translate(token))
  print("Перевод:", ' '.join(translation))

Оригинальное предложение: All dies ist im Sinne der Subsidiarität außerordentlich zu begrüßen .
Перевод: upheld done is in a the subsidiarity  to welcomed .
Оригинальное предложение: Sehr geehrte Kollegen ! Ich danke Frau Schroedter für den fundierten Bericht .
Перевод: year and and , I  Madam  to the  report .
Оригинальное предложение: Das strikte Verbot aller Beihilfen , die nicht vom Kodex gedeckt sind , kann nur so durchgesetzt werden .
Перевод: the    aid , the not the   are , of . the no-smoking , .
Оригинальное предложение: Diese Eigeninitiative darf nicht von europäischer Ebene aus zerstört werden .
Перевод: This   not the  economies are  , .
Оригинальное предложение: Wir haben eine sehr interessante Debatte vor uns .
Перевод: We have a .  extensive and the .
Оригинальное предложение: Das ist der Fall von Alexander Nikitin .
Перевод: the is the , the   .
Оригинальное предложение: Rahmenbedingungen sind zur Verhinderung von Mißbrauch unerläßlich , zum Beispiel durch das Kartellr

Проверяем его на грамматичность

In [111]:
from nltk.translate.bleu_score import corpus_bleu

reference = [X_test_tokens[0]]  # Или любое другое предложение

candidate = [translate(token) for token in y_test_tokens[0]]

bleu_score = corpus_bleu([reference], [candidate])

print("BLEU Score:", bleu_score)

BLEU Score: 4.305320836691481e-155


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


In [114]:
reference

[['All',
  'of',
  'this',
  'is',
  'in',
  'accordance',
  'with',
  'the',
  'principle',
  'of',
  'subsidiarity',
  'and',
  'is',
  'therefore',
  'to',
  'be',
  'greatly',
  'welcomed',
  '.']]

In [115]:
candidate

['upheld',
 'done',
 'is',
 'in',
 'a',
 'the',
 'subsidiarity',
 '',
 'to',
 'welcomed',
 '.']